Aluno Flavio Amorim

Professora Carolina

# Preparando o ambiente

In [1]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5         # tamanho do individuo na inicializacao
MAX_CUSTOMER = 10         # quantidade maxima de pedidos a serem tratados


In [3]:
MAX_VALOR  = 500000  ### valor maximo transportado no 
MAX_VOLUME = 4500    ### volume maximo ocupado no caminhao
MAX_HORAS = 6        ### nao pode trabalhar mais de 6 horas por dia

# variaveis auxiliares (banco de dados) 

In [4]:
import pandas as pd


In [5]:
# iniciar as variaveis
solicitacao_tabela = { 'id':     [0,1,2,3,4,5,6,7,8,9],                                             ## customer order id
                       'urgente':[1,0,1,1,0,0,0,0,1,1],                                             ## indicativo de urgencia
                       'new':    [0,1,1,0,1,1,1,1,0,0],                                             ## indicativo de cliente novo
                       'valor':  [100000,10000,100,200000,400000,50000,10000,500,3000,10000],       ## value
                       'volume': [1000, 2000,300,400,500,600,700,800,900,3000],                     ## volume
                       'bairro': [1,2,3,4,5,6,7,8,9,1]}                                             ## area in town
df_solicitacao = pd.DataFrame(solicitacao_tabela)
df_solicitacao.head(10)

,id,urgente,new,valor,volume,bairro
0,0,1,0,100000,1000,1
1,1,0,1,10000,2000,2
2,2,1,1,100,300,3
3,3,1,0,200000,400,4
4,4,0,1,400000,500,5
5,5,0,1,50000,600,6
6,6,0,1,10000,700,7
7,7,0,1,500,800,8
8,8,1,0,3000,900,9
9,9,1,0,10000,3000,1


In [6]:
caminhao_tabela = { 'id':        [1,2],
                    'opex':      [1000,5000],
                    'capacidade':[500,5000]}
df_caminhao = pd.DataFrame(caminhao_tabela)
df_caminhao.head()

,id,opex,capacidade
0,1,1000,500
1,2,5000,5000


In [8]:
distancia_tabela ={
                    'destino': [0,  20, 10, 15, 20, 10, 5,  20, 40, 10],
                    'dest1':   [20, 0,  5,  25, 25, 5,  25,  5, 10, 30],
                    'dest2':   [10, 5,  0,  80, 30, 30, 40, 60, 50, 30],
                    'dest3':   [15, 25, 80,  0, 15, 25, 60, 30, 30, 20],
                    'dest4':   [20, 25, 30, 15, 0,  50, 20, 25, 15, 10],
                    'dest5':   [10, 5,  30, 25, 50, 0,  20, 40, 40, 50],
                    'dest6':   [5,  25, 40, 60, 20, 20,  0, 40, 20, 30],
                    'dest7':   [20, 5,  60, 30, 25, 40, 40,  0, 50, 30],
                    'dest8':   [40, 10, 50, 30, 15, 40, 20, 50,  0, 20],
                    'dest9':  [10, 30, 30, 20, 10, 50, 30, 30, 20,  0]}

df_distancia = pd.DataFrame(distancia_tabela)
df_distancia


,destino,dest1,dest2,dest3,dest4,dest5,dest6,dest7,dest8,dest9
0,0,20,10,15,20,10,5,20,40,10
1,20,0,5,25,25,5,25,5,10,30
2,10,5,0,80,30,30,40,60,50,30
3,15,25,80,0,15,25,60,30,30,20
4,20,25,30,15,0,50,20,25,15,10
5,10,5,30,25,50,0,20,40,40,50
6,5,25,40,60,20,20,0,40,20,30
7,20,5,60,30,25,40,40,0,50,30
8,40,10,50,30,15,40,20,50,0,20
9,10,30,30,20,10,50,30,30,20,0


In [9]:
fator_velocidade   = 25       ## velocidade media é 25km/hora
fator_custo   = 20            ## R$20,00 por km rodado
fator_seguro  = 0.1           ## seguro custa 10% do valor transportado
centro_distribuicao = 0       ## centro de distribuicao é a localidade 0 

# Iniciando a estrutura de GA

In [10]:
def funcao_objetivo(individual):
    val = 0.0          ## valor transportado
    custo = 0.0        ## custo para empresa  - MINIMIZAR
    volume = 0.0       ## volume a transportar - MAXIMIZAR
    HORAS = 0.0        ## nao pode exceder 8hs

    ##variáveis auxiliares 
    distancia = 0.0
    ponto_a = 99
    ponto_b = 99
    for idx,item in enumerate(individual):
        val += int(df_solicitacao[df_solicitacao.id == int(item)]['valor'])
        volume += int(df_solicitacao[df_solicitacao.id == int(item)]['volume'])

        if idx == 0:                      ## no inicio da jornada a origem é o CD
            ponto_a = 0
        if idx == len(individual)-1:      ## no final da jornada, o destino é o CD
            ponto_b = 0  
        else:    
            ponto_b = int(df_solicitacao[df_solicitacao.id == item]['bairro'])

        distancia += df_distancia.iloc[ponto_a,ponto_b]
        ponto_a = ponto_b                ## no proximo pickup, o local de origem é o destino do anterior
    
    custo_deslocamento = distancia * fator_custo
    custo_seguro = val * fator_seguro
    custo_opex = 0
    custo = custo_deslocamento + custo_seguro + custo_opex

    horas = distancia / fator_velocidade

    if len(individual) > MAX_CUSTOMER or volume > MAX_VOLUME or horas > MAX_HORAS:     ### se extrapolou essas 3 restricoes
        return 100000, 0                                                   ###penalizo com  custo elevado e volume de carga = 0

    return custo, volume

def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(MAX_CUSTOMER))  # aqui
    return individual,

#Restricoes

In [11]:
# Avaliando validade do indivíduo

def feasible_new(individual):
    new = 0.0
    for item in individual:
        new += int(df_solicitacao[df_solicitacao.id == int(item)]['new'])
    if new < MAX_CUSTOMER:                # nem todos sao novos
        return True                   
                                      
    return False


In [12]:
def feasible_urgencia(individual):
    urgencia = 0.0
    for item in individual:
        urgencia += int(df_solicitacao[df_solicitacao.id == int(item)]['urgente'])
    if urgencia == 0:                # nao atendeu urgencia
        return False                   
                                      
    return True

# Criando a estrutura GA

In [13]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))           
                                                                       ### custo total:         minimizar
                                                                       ### valor transportado:  maximizar
                                                                       
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, MAX_CUSTOMER)             ### gerador randomico   aqui

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [14]:
toolbox.register("evaluate", funcao_objetivo)
toolbox.decorate('evaluate', tools.DeltaPenalty(feasible_urgencia, (10000,-10000))) # deixou de atender pedidos com urgencia 
toolbox.decorate('evaluate', tools.DeltaPenalty(feasible_new, (10000,-10000)))  
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [34]:
def main():
    random.seed(64)
    NGEN = 75
    MU = 50
    LAMBDA = 100
    CXPB = 0.8
    MUTPB = 0.2

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof,verbose=True)

    return pop, stats, hof

In [35]:
if __name__ == "__main__":
    id_caminhao = int(df_caminhao[df_caminhao.id==1]['id'])  	##id 	opex 	capacidade
    p, s, h = main()    

gen	nevals	avg            	std                            	min              	max              
0  	50    	[59193.  1350.]	[34874.96639425  2751.38147119]	[  2960. -10000.]	[100000.   4300.]
1  	100   	[18354.2  2658. ]	[15633.09516251  1007.39068886]	[ 10. 300.]      	[53910.  4300.]  
2  	100   	[5914.8 1812. ]  	[10362.7703323   1406.36268437]	[ 10. 300.]      	[37500.  4300.]  
3  	100   	[1815.4  874. ]  	[6698.96535593 1082.55438662]  	[ 10. 300.]      	[33100.  4300.]  
4  	100   	[1806.6  834. ]  	[6701.83873575 1094.82601357]  	[ 10. 300.]      	[33100.  4300.]  
5  	100   	[1806.6  834. ]  	[6701.83873575 1094.82601357]  	[ 10. 300.]      	[33100.  4300.]  
6  	100   	[1806.6  834. ]  	[6701.83873575 1094.82601357]  	[ 10. 300.]      	[33100.  4300.]  
7  	100   	[1806.6  834. ]  	[6701.83873575 1094.82601357]  	[ 10. 300.]      	[33100.  4300.]  
8  	100   	[2468.4  914. ]  	[7999.85283865 1194.48901209]  	[ 10. 300.]      	[33100.  4300.]  
9  	100   	[3792. 1074.]    	[9976

In [109]:
# para testes
#h.items

In [110]:
# para testes
#h.keys

# Melhor solução

In [36]:
# Melhor solução nesta iteração
print('Melhor indivíduo: ')
print(h[-1])

Melhor indivíduo: 
Individual({8, 9, 5})


In [37]:
funcao_objetivo(h[-1])

(7500.0, 4500.0)

# Atendimento das restrições e prioridades

In [22]:
val_num = 0
vol_num = 0
new_num = 0
urg_num = 0

for id in h[-1]:
    val_num += int(df_solicitacao[df_solicitacao.id == int(id)]['valor'])
    vol_num += int(df_solicitacao[df_solicitacao.id == int(id)]['volume'])
    new_num += int(df_solicitacao[df_solicitacao.id == int(id)]['new'])
    urg_num += int(df_solicitacao[df_solicitacao.id == int(id)]['urgente'])

print(f'numero de clientes novos atendidos {new_num}')
print(f'volume transportado                {vol_num}')
print(f'valor transportados                {val_num}')
print(f'numero de pedidos urgentes         {urg_num}')



numero de clientes novos atendidos 1
volume transportado                4500
valor transportados                63000
numero de pedidos urgentes         2


In [23]:
print("(custo , volume)")
funcao_objetivo(h[-1])

(custo , volume)


(7500.0, 4500.0)